# Azure Foundry IQ - Complete Knowledge Base Guide

## 📋 Overview

**Foundry IQ** is the brand name for Azure AI Search's Agentic Retrieval capability, designed specifically for RAG (Retrieval Augmented Generation) and Agent grounding scenarios. It implements the separation of **domain knowledge** from **Agent logic**, allowing multiple Agents to share the same knowledge base.

### 🔑 Core Concepts

```mermaid
flowchart TB
    subgraph KB["Knowledge Base"]
        direction TB
        LLM["🤖 LLM Connection Config"]
        RI["📝 Retrieval Instructions"]
        AI["📝 Answer Instructions"]
        OM["⚙️ Output Mode / Reasoning Effort"]
    end

    subgraph Sources["Knowledge Sources"]
        KS1["📂 Search Index"]
        KS2["📂 Azure Blob"]
        KS3["📂 SharePoint"]
        KS4["📂 OneLake"]
        KS5["🌐 Web (Bing)"]
    end

    subgraph Indexes["Search Indexes"]
        IDX1[(Index A)]
        IDX2[(Index B)]
    end

    KB -->|"references"| Sources
    KS1 --> IDX1
    KS2 --> IDX2
```

### 📊 Output Mode and Reasoning Effort

| Output Mode | Description | Use Case |
|-------------|-------------|----------|
| `ANSWER_SYNTHESIS` | LLM generates natural language answers with citations | Direct user-facing Q&A |
| `EXTRACTED_DATA` | Returns raw search results | Agent integration, letting Agent control response format |

&nbsp;

| Reasoning Effort | Description | Latency | Cost |
|------------------|-------------|---------|------|
| `minimal` | Skips LLM query planning | ⚡ Lowest | 💰 Lowest |
| `low` (default) | Single LLM query planning | Medium | Medium |
| `medium` | Iterative query optimization | Higher | Higher |

---

## 📑 Table of Contents

| Step | Content | Description |
|------|---------|-------------|
| [Step 1](#step-1-configure-environment-variables) | Configure Environment Variables | Set up Azure AI Search and Azure OpenAI connections |
| [Step 2](#step-2-create-search-index-with-agentic-retrieval-support) | Create Search Index | Configure vector search, semantic search, vectorizer |
| [Step 3](#step-3-upload-sample-documents) | Upload Documents | Generate embeddings and index documents |
| [Step 4](#step-4-create-knowledge-source) | Create Knowledge Source | Define data source reference and field mappings |
| [Step 5](#step-5-create-knowledge-base) | Create Knowledge Base | Configure LLM, output mode, retrieval instructions |
| [Step 6](#step-6-query-knowledge-base-agentic-retrieval) | Query Knowledge Base | Use Agentic Retrieval for Q&A |
| [Step 7](#step-7-view-and-manage-resources) | Manage Resources | List created KB and KS |
| [Step 8](#step-8-foundry-agent-service-integration-advanced) | Agent Integration | Foundry Agent Service integration example |

---

## 📚 Reference Documentation

- [Agentic Retrieval Overview](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-overview)
- [Create Knowledge Base](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-how-to-create-knowledge-base)
- [Knowledge Source Types](https://learn.microsoft.com/en-us/azure/search/agentic-knowledge-source-overview)
- [Foundry Agent Service Integration](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/knowledge-retrieval)

In [ ]:
# Install required packages (using latest preview version)
%pip install azure-search-documents azure-identity python-dotenv openai requests --pre -qU

In [ ]:
import os
import json
import requests
from pathlib import Path
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SearchIndexKnowledgeSource,
    SearchIndexKnowledgeSourceParameters,
    SearchIndexFieldReference,
    KnowledgeBase,
    KnowledgeBaseAzureOpenAIModel,
    KnowledgeSourceReference,
    KnowledgeRetrievalOutputMode,
    KnowledgeRetrievalLowReasoningEffort,
    KnowledgeRetrievalMinimalReasoningEffort,
    KnowledgeRetrievalMediumReasoningEffort
)

# Load environment variables
load_dotenv()

print("✅ Packages imported successfully")

## Step 1: Configure Environment Variables

Set up Azure AI Search and Azure OpenAI connection information.

In [ ]:
# ============================================
# Configuration Parameters
# ============================================

# Azure AI Search Configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")  # e.g., https://your-search.search.windows.net
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")    # Admin key

# Azure OpenAI Configuration
aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")    # e.g., https://your-openai.openai.azure.com
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")

# Model deployment names (read from environment variables)
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "embedding")
embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-ada-002")
embedding_dimensions = 1536  # text-embedding-ada-002 dimensions

gpt_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini")
gpt_model = gpt_deployment

# Resource naming
index_name = "foundry-iq-demo-index"
knowledge_source_name = "foundry-iq-knowledge-source"
knowledge_base_name = "foundry-iq-knowledge-base"

# Verify configuration
print("📋 Configuration Check:")
print(f"   Search Endpoint: {search_endpoint}")
print(f"   OpenAI Endpoint: {aoai_endpoint}")
print(f"   Embedding Model: {embedding_deployment}")
print(f"   GPT Model: {gpt_deployment}")
print(f"   Index Name: {index_name}")
print(f"   Knowledge Source: {knowledge_source_name}")
print(f"   Knowledge Base: {knowledge_base_name}")

In [ ]:
# Initialize clients
# Choose authentication method: API Key or RBAC

# Method 1: Using API Key (simple but not recommended for production)
credential = AzureKeyCredential(search_api_key)

# Method 2: Using RBAC (recommended for production)
# credential = DefaultAzureCredential()

# Create index client
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=credential
)

print("✅ Client initialized successfully")

## Step 2: Create Search Index with Agentic Retrieval Support

The index needs to include:
- Text fields (for keyword search and semantic ranking)
- Vector fields (for vector search)
- Vectorizer (for automatic query vectorization)
- Semantic configuration (for semantic reranking)

In [ ]:
# Create index definition
index = SearchIndex(
    name=index_name,
    fields=[
        # Primary key
        SearchField(
            name="id",
            type="Edm.String",
            key=True,
            filterable=True,
            sortable=True
        ),
        # Document title
        SearchField(
            name="title",
            type="Edm.String",
            searchable=True,
            filterable=True,
            sortable=True
        ),
        # Document content (for search and semantic ranking)
        SearchField(
            name="content",
            type="Edm.String",
            searchable=True,
            filterable=False,
            sortable=False
        ),
        # Vector field
        SearchField(
            name="content_vector",
            type="Collection(Edm.Single)",
            stored=False,  # Don't store to save space
            vector_search_dimensions=embedding_dimensions,
            vector_search_profile_name="vector_profile"
        ),
        # Metadata fields
        SearchField(
            name="source",
            type="Edm.String",
            filterable=True,
            facetable=True
        ),
        SearchField(
            name="page_number",
            type="Edm.Int32",
            filterable=True,
            sortable=True
        ),
        SearchField(
            name="category",
            type="Edm.String",
            filterable=True,
            facetable=True
        )
    ],
    # Vector search configuration
    vector_search=VectorSearch(
        profiles=[
            VectorSearchProfile(
                name="vector_profile",
                algorithm_configuration_name="hnsw_config",
                vectorizer_name="azure_openai_vectorizer"  # Auto-vectorize at query time
            )
        ],
        algorithms=[
            HnswAlgorithmConfiguration(
                name="hnsw_config",
                # Optional parameters: m=4, ef_construction=400, ef_search=500
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=aoai_endpoint,
                    deployment_name=embedding_deployment,
                    model_name=embedding_model,
                    api_key=aoai_api_key
                )
            )
        ]
    ),
    # Semantic search configuration (must enable semantic ranking)
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    title_field=SemanticField(field_name="title"),
                    content_fields=[SemanticField(field_name="content")]
                )
            )
        ]
    )
)

# Create or update index
index_client.create_or_update_index(index)
print(f"✅ Index '{index_name}' created successfully")

## Step 3: Upload Sample Documents

Upload some sample documents to the index. In real scenarios, you can:
- Extract text from PDF, Word, and other documents
- Use Azure Document Intelligence to process complex documents
- Use Indexer to automatically pull data from Blob, SharePoint, etc.

In [ ]:
from openai import AzureOpenAI

# Initialize Azure OpenAI client for generating embeddings
aoai_client = AzureOpenAI(
    api_key=aoai_api_key,
    api_version="2024-06-01",
    azure_endpoint=aoai_endpoint
)

def generate_embedding(text: str) -> list[float]:
    """Generate vector embedding for text"""
    response = aoai_client.embeddings.create(
        input=text,
        model=embedding_deployment
    )
    return response.data[0].embedding

# Sample documents (about Azure AI services)
sample_documents = [
    {
        "id": "doc1",
        "title": "Azure AI Search Overview",
        "content": "Azure AI Search is an enterprise-grade search service that provides vector search, full-text search, and semantic search capabilities. It supports Agentic Retrieval and can integrate with LLMs for intelligent retrieval. Key features include: vector search supporting multiple algorithms (HNSW, exhaustive KNN), semantic reranking for improved search relevance, and hybrid search combining keyword and vector search advantages.",
        "source": "azure-docs",
        "page_number": 1,
        "category": "search"
    },
    {
        "id": "doc2",
        "title": "Foundry IQ Introduction",
        "content": "Foundry IQ is the brand name for Azure AI Search's Agentic Retrieval capability. It creates a separation between domain knowledge and Agent logic, supporting large-scale RAG and grounding. Through the concepts of Knowledge Base and Knowledge Source, you can independently manage knowledge content, and multiple Agents can share the same knowledge base.",
        "source": "azure-docs",
        "page_number": 2,
        "category": "ai"
    },
    {
        "id": "doc3",
        "title": "Knowledge Base Configuration",
        "content": "Knowledge Base is the core of the Agentic Retrieval pipeline. It defines which knowledge sources to query and the default behavior of retrieval operations. Key properties include: name (unique identifier), description (used by LLM for query planning), knowledge_sources (one or more knowledge source references), models (LLM connection info), output_mode (answer_synthesis or extracted_data), retrieval_reasoning_effort (minimal, low, medium).",
        "source": "azure-docs",
        "page_number": 3,
        "category": "configuration"
    },
    {
        "id": "doc4",
        "title": "Knowledge Source Types",
        "content": "Azure AI Search supports multiple types of Knowledge Sources: 1) searchIndex - wraps existing indexes, 2) azureBlob - generates indexer pipeline pulling from Blob containers, 3) indexedOneLake - pulls from OneLake, 4) indexedSharePoint - pulls from SharePoint sites, 5) remoteSharePoint - directly queries SharePoint, 6) webParameters - gets real-time grounding data from Bing.",
        "source": "azure-docs",
        "page_number": 4,
        "category": "configuration"
    },
    {
        "id": "doc5",
        "title": "Retrieval Reasoning Effort",
        "content": "Retrieval Reasoning Effort determines the level of LLM-related query processing. minimal - bypasses LLM query planning, reducing cost and latency; low (default) - balanced LLM processing approach; medium - maximizes LLM processing for improved relevance. For Foundry Agent Service integration, minimal is recommended for better performance and lower cost.",
        "source": "azure-docs",
        "page_number": 5,
        "category": "performance"
    },
    {
        "id": "doc6",
        "title": "MCP Tool Integration",
        "content": "Model Context Protocol (MCP) is the standard protocol for Agent to Knowledge Base communication. In Foundry Agent Service, you can create a RemoteTool connection using the project's managed identity to connect to the Knowledge Base's MCP endpoint. The Agent calls the knowledge_base_retrieve MCP tool at runtime to retrieve relevant content and generate responses with citations.",
        "source": "azure-docs",
        "page_number": 6,
        "category": "integration"
    }
]

# Generate vector embeddings for each document
print("🔄 Generating vector embeddings...")
for doc in sample_documents:
    doc["content_vector"] = generate_embedding(doc["content"])
    print(f"   ✓ {doc['title']}")

print(f"\n✅ Embeddings generated for all {len(sample_documents)} documents")

In [ ]:
# Upload documents to index
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=credential
)

result = search_client.upload_documents(documents=sample_documents)
succeeded = sum(1 for r in result if r.succeeded)
print(f"✅ Successfully uploaded {succeeded}/{len(sample_documents)} documents to index '{index_name}'")

## Step 4: Create Knowledge Source

Knowledge Source is a reusable reference to source data. It specifies:
- Target index
- Which fields to include in references
- Optional description (helps LLM with knowledge source selection)

In [ ]:
# Create Knowledge Source
knowledge_source = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for Azure AI service documentation, including Azure AI Search, Foundry IQ, Knowledge Base configuration, etc.",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        # Specify fields to include in references (avoid large fields like vectors)
        source_data_fields=[
            SearchIndexFieldReference(name="id"),
            SearchIndexFieldReference(name="title"),
            SearchIndexFieldReference(name="source"),
            SearchIndexFieldReference(name="page_number"),
            SearchIndexFieldReference(name="category")
        ]
    )
)

# Create or update Knowledge Source
index_client.create_or_update_knowledge_source(knowledge_source=knowledge_source)
print(f"✅ Knowledge Source '{knowledge_source_name}' created successfully")

## Step 5: Create Knowledge Base

Knowledge Base is the core of the Agentic Retrieval pipeline. It:
- Connects one or more Knowledge Sources
- Configures LLM for query planning and answer synthesis
- Defines output mode and retrieval behavior

### Output Mode Options
- `ANSWER_SYNTHESIS`: LLM generates natural language answers with citations
- `EXTRACTED_DATA`: Returns raw search results for downstream Agent processing

### Reasoning Effort Options
- `minimal`: Skips LLM query planning, lowest latency and cost
- `low` (default): Balanced LLM processing
- `medium`: Maximizes LLM processing for improved relevance

In [ ]:
# Configure Azure OpenAI connection parameters
aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=aoai_endpoint,
    deployment_name=gpt_deployment,
    model_name=gpt_model,
    api_key=aoai_api_key
)

# Create Knowledge Base
knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    description="Azure AI services knowledge base for answering questions about Azure AI Search, Foundry IQ, and Agentic Retrieval",
    
    # Retrieval instructions: Guide LLM on how to select knowledge sources and formulate queries
    retrieval_instructions="Use this knowledge source to answer questions about Azure AI services. If the question involves search, knowledge bases, or Agent integration, use this knowledge source.",
    
    # Answer instructions: Guide LLM on how to format answers
    answer_instructions="Provide clear, accurate answers based on retrieved documents. Cite source document titles and page numbers when relevant.",
    
    # Output mode
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    
    # Knowledge source list
    knowledge_sources=[
        KnowledgeSourceReference(name=knowledge_source_name)
    ],
    
    # LLM model configuration
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    
    # Retrieval reasoning effort level
    # Use minimal for lower latency and cost (recommended for Foundry Agent Service integration)
    # Use low or medium for better query planning and relevance
    retrieval_reasoning_effort=KnowledgeRetrievalLowReasoningEffort
)

# Create or update Knowledge Base
index_client.create_or_update_knowledge_base(knowledge_base)
print(f"✅ Knowledge Base '{knowledge_base_name}' created successfully")

## Step 6: Query Knowledge Base (Agentic Retrieval)

Use `KnowledgeBaseRetrievalClient` to send query requests to the knowledge base.

Queries support:
- Multi-turn conversations (via messages array)
- Custom knowledge source parameters
- Activity logs (for debugging and auditing)

In [ ]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import (
    KnowledgeBaseRetrievalRequest,
    KnowledgeBaseMessage,
    KnowledgeBaseMessageTextContent,
    SearchIndexKnowledgeSourceParams
)

# Initialize retrieval client
kb_client = KnowledgeBaseRetrievalClient(
    endpoint=search_endpoint,
    knowledge_base_name=knowledge_base_name,
    credential=credential
)

def query_knowledge_base(question: str, conversation_history: list = None):
    """
    Send a query to the Knowledge Base and return results
    
    Args:
        question: User question
        conversation_history: Optional conversation history
    
    Returns:
        Retrieval results including answer and citations
    """
    # Build message list
    messages = []
    
    # Add conversation history (if any)
    if conversation_history:
        for msg in conversation_history:
            messages.append(
                KnowledgeBaseMessage(
                    role=msg["role"],
                    content=[KnowledgeBaseMessageTextContent(text=msg["content"])]
                )
            )
    
    # Add current question
    messages.append(
        KnowledgeBaseMessage(
            role="user",
            content=[KnowledgeBaseMessageTextContent(text=question)]
        )
    )
    
    # Build retrieval request
    request = KnowledgeBaseRetrievalRequest(
        messages=messages,
        knowledge_source_params=[
            SearchIndexKnowledgeSourceParams(
                knowledge_source_name=knowledge_source_name,
                include_references=True,           # Include reference information
                include_reference_source_data=True, # Include source data details
                always_query_source=True           # Always query this knowledge source
            )
        ],
        include_activity=True  # Include activity logs (for debugging)
    )
    
    # Execute retrieval
    result = kb_client.retrieve(retrieval_request=request)
    
    return result

print("✅ Query function ready")

In [ ]:
# Test query 1: Basic question
question = "What is Foundry IQ? What are its advantages?"

print(f"❓ Question: {question}")
print("=" * 60)

result = query_knowledge_base(question)

# Display answer
if result.response:
    for resp in result.response:
        for content in resp.content:
            print(f"\n📝 Answer:\n{content.text}")

# Display citations
if result.references:
    print(f"\n📚 References ({len(result.references)}):")
    for i, ref in enumerate(result.references[:5], 1):
        ref_dict = ref.as_dict()
        source_data = ref_dict.get("sourceData", {})
        print(f"   {i}. {source_data.get('title', 'N/A')} (Page {source_data.get('page_number', 'N/A')})")

In [ ]:
# Test query 2: Technical details
question = "What Retrieval Reasoning Effort levels does Knowledge Base support? What are their characteristics?"

print(f"❓ Question: {question}")
print("=" * 60)

result = query_knowledge_base(question)

# Display answer
if result.response:
    for resp in result.response:
        for content in resp.content:
            print(f"\n📝 Answer:\n{content.text}")

# Display citations
if result.references:
    print(f"\n📚 References ({len(result.references)}):")
    for i, ref in enumerate(result.references[:5], 1):
        ref_dict = ref.as_dict()
        source_data = ref_dict.get("sourceData", {})
        print(f"   {i}. {source_data.get('title', 'N/A')} (Page {source_data.get('page_number', 'N/A')})")

In [ ]:
# Test query 3: Multi-turn conversation
conversation = [
    {"role": "user", "content": "What are the main features of Azure AI Search?"},
    {"role": "assistant", "content": "Azure AI Search provides vector search, full-text search, and semantic search capabilities, supporting Agentic Retrieval."}
]

follow_up = "How does it integrate with LLMs?"

print(f"💬 Conversation history: {len(conversation)} turns")
print(f"❓ Follow-up: {follow_up}")
print("=" * 60)

result = query_knowledge_base(follow_up, conversation)

# Display answer
if result.response:
    for resp in result.response:
        for content in resp.content:
            print(f"\n📝 Answer:\n{content.text}")

## Step 7: View and Manage Resources

List the created Knowledge Bases and Knowledge Sources.

In [ ]:
# List all Knowledge Bases
print("📋 Created Knowledge Bases:")
print("-" * 40)

# Use REST API to list Knowledge Bases
kb_list_url = f"{search_endpoint}/knowledgebases"
params = {"api-version": "2025-11-01-preview", "$select": "name,description"}
headers = {"api-key": search_api_key}

response = requests.get(kb_list_url, params=params, headers=headers)
if response.status_code == 200:
    kbs = response.json().get("value", [])
    for kb in kbs:
        print(f"   • {kb.get('name')}: {kb.get('description', 'No description')}")
else:
    print(f"   ⚠️ Unable to get list: {response.status_code}")

In [ ]:
# List all Knowledge Sources
print("📋 Created Knowledge Sources:")
print("-" * 40)

ks_list_url = f"{search_endpoint}/knowledgesources"
params = {"api-version": "2025-11-01-preview", "$select": "name,description"}

response = requests.get(ks_list_url, params=params, headers=headers)
if response.status_code == 200:
    sources = response.json().get("value", [])
    for ks in sources:
        print(f"   • {ks.get('name')}: {ks.get('description', 'No description')}")
else:
    print(f"   ⚠️ Unable to get list: {response.status_code}")

## Step 8: Foundry Agent Service Integration (Advanced)

The following code demonstrates how to integrate Knowledge Base with Microsoft Foundry Agent Service.

### Prerequisites
- Microsoft Foundry project
- Project managed identity configuration
- Proper RBAC role assignments

In [ ]:
# Foundry Agent Service integration example (requires additional configuration)

# Note: The following code requires a configured Microsoft Foundry project
# and the azure-ai-projects package installed

# %pip install azure-ai-projects --pre -qU

FOUNDRY_INTEGRATION_EXAMPLE = """
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Configuration
project_endpoint = "https://your-project.services.ai.azure.com/api/projects/your-project-name"
knowledge_base_mcp_endpoint = f"{search_endpoint}/knowledgebases/{knowledge_base_name}/mcp"

# Initialize project client
credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

# Create project connection (for MCP communication)
# This needs to be created via Azure Management API as a RemoteTool connection

# Create Agent with MCP tools
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="knowledge-retrieval-agent",
    instructions='''You are a helpful assistant that must use the knowledge base to answer all questions.
Every answer must provide annotations using the MCP knowledge base tool.
If you cannot find the answer, respond with "I don't know".''',
    tools=[{
        "type": "mcp",
        "mcp": {
            "server_label": "AzureAISearch",
            "allowed_tools": ["knowledge_base_retrieve"]
        }
    }],
    tool_resources={
        "mcp": {
            "connections": {
                "AzureAISearch": {
                    "connection_string": "your-project-connection-name"
                }
            }
        }
    }
)

print(f"Agent created: {agent.name}")
"""

print("📘 Foundry Agent Service integration example code:")
print(FOUNDRY_INTEGRATION_EXAMPLE)

## Clean Up Resources (Optional)

⚠️ **Warning**: Running the following code will delete all created resources!

In [ ]:
# ⚠️ Dangerous Operation - Delete Resources ⚠️
# Uncomment the following code to execute deletion

# confirm = input("Enter 'DELETE' to confirm deletion of all resources: ")
# if confirm == "DELETE":
#     # 1. Delete Knowledge Base (must delete first as it references Knowledge Source)
#     try:
#         index_client.delete_knowledge_base(knowledge_base_name)
#         print(f"✅ Knowledge Base '{knowledge_base_name}' deleted")
#     except Exception as e:
#         print(f"⚠️ Failed to delete Knowledge Base: {e}")
#     
#     # 2. Delete Knowledge Source
#     try:
#         index_client.delete_knowledge_source(knowledge_source=knowledge_source_name)
#         print(f"✅ Knowledge Source '{knowledge_source_name}' deleted")
#     except Exception as e:
#         print(f"⚠️ Failed to delete Knowledge Source: {e}")
#     
#     # 3. Delete index (optional, don't delete if index is used for other purposes)
#     try:
#         index_client.delete_index(index_name)
#         print(f"✅ Index '{index_name}' deleted")
#     except Exception as e:
#         print(f"⚠️ Failed to delete Index: {e}")
#     
#     print("\n✅ All resources cleaned up!")
# else:
#     print("Deletion cancelled")

## Summary

This notebook demonstrated the complete Azure Foundry IQ workflow:

1. **Create Search Index** - Configure vector search, semantic search, and vectorizer
2. **Upload Documents** - Generate embeddings and index documents
3. **Create Knowledge Source** - Define data source reference
4. **Create Knowledge Base** - Configure retrieval pipeline and LLM connection
5. **Query Knowledge Base** - Use Agentic Retrieval for Q&A

## Related Resources

- [Agentic Retrieval Overview](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-overview)
- [Create Knowledge Base](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-how-to-create-knowledge-base)
- [Knowledge Source Types](https://learn.microsoft.com/en-us/azure/search/agentic-knowledge-source-overview)
- [Foundry Agent Service Integration](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/knowledge-retrieval)
- [GitHub Sample Code](https://github.com/Azure-Samples/azure-search-python-samples/tree/main/agentic-retrieval-pipeline-example)

## Best Practices

1. **Choose the right Reasoning Effort**:
   - Use `minimal` when integrating with Foundry Agent Service
   - Use `low` or `medium` when complex query planning is needed

2. **Choose the right Output Mode**:
   - `extracted_data`: Let Agent control response format (recommended for Agent integration)
   - `answer_synthesis`: Get formatted answers directly

3. **Security**:
   - Use RBAC instead of API Key in production
   - Configure managed identity for service-to-service authentication

4. **Performance Optimization**:
   - Use vectorizer for automatic query vectorization
   - Set `source_data_fields` appropriately to avoid returning large fields